In [2]:
import pandas as pd

# Load the Excel file
file_path = "data.xlsx"  # Change this to the path of your Excel file
questions_df = pd.read_excel(file_path)
questions_df.sample(frac=1)

# Initialize the score
score = 0

# List to keep track of asked questions
asked_questions = []

# Function to get a random question that hasn't been asked correctly before
def get_random_question():
    available_questions = questions_df[~questions_df["Questions"].isin(asked_questions)]
    if available_questions.empty:
        return None
    return available_questions.sample().iloc[0]

# Ask 10 questions
for i in range(10):
    question = get_random_question()

    if question is None:
        print("No more available questions.")
        break
    a=0
    while True:
        print(f"Question {i + 1}: {question['Questions']}")
        user_answer = input("Your answer: ")
        correct_answer = question["Answer"]

        if user_answer.strip() == correct_answer.strip():
            print("Correct!")
            if question["Questions"] not in asked_questions and a==0:
                score += 1
                asked_questions.append(question["Questions"])
                a=0
            break
        else:
            print(f"Sorry, the correct answer is: {correct_answer}")
            a=1

print(f"\nGame Over! Your final score is: {score} out of 10.")


Question 1: To check if the word has any special character
Your answer: word.isdecimal()
Sorry, the correct answer is: word.isalnum()
Question 1: To check if the word has any special character
Your answer: word.isalnum()
Correct!
Question 2: Filter a df by multiple categories a, b, c present in a particular column A
Your answer: df[df['A'].isin['a','b','c']]
Sorry, the correct answer is: df[df['A'].isin(['a','b','c'])]
Question 2: Filter a df by multiple categories a, b, c present in a particular column A
Your answer: df[df['A'].isin(['a','b','c'])
Sorry, the correct answer is: df[df['A'].isin(['a','b','c'])]
Question 2: Filter a df by multiple categories a, b, c present in a particular column A
Your answer: df[df('A').isin(['a','b','c'])]
Sorry, the correct answer is: df[df['A'].isin(['a','b','c'])]
Question 2: Filter a df by multiple categories a, b, c present in a particular column A
Your answer: df[df['A'].isin(['a','b','c'])]
Correct!
Question 3: Randomly selects 10 values from a 